In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
from xml.etree.ElementPath import get_parent_map
import pandas as pd
from IPython.display import display

import sys
sys.path.append(str(Path().resolve().parents[1]/'gepard'))

import pyximport  # TODO: tried this to be able to import gepard.dataset, but it didn't work
pyximport.install()

import gepard.dataset as dataset

In [ ]:
data_root = Path('/run/media/rob/temp_data/Schleiproben_test')

In [ ]:
if Path('pkl_files.csv').is_file():
    pkls = pd.read_csv('pkl_files.csv')

else:
    pkls = []
    for path in data_root.rglob('*.pkl'):
        pkls.append({
            'path': path.parent,
            'name': path.name,
            'tmod': path.stat().st_mtime,
            'size': path.stat().st_size
        })
    pkls = pd.DataFrame(pkls)
    pkls.to_csv('pkl_files.csv', index=False)

In [ ]:
# Routine for manual decision of which pkl files should be used
# TODO: not functional yet

# pkls['use_this'] = False

# for path, group in pkls.head(10).groupby(['path'], sort=False):
#     if len(group) > 1:
#         display(group[['name', 'tmod', 'size']].sort_values(by='tmod', ascending=False))
#         while True:
#             try:
#                 num = int(input("Which one do you want to keep? "))
#             except ValueError:
#                 print("Sorry, I didn't understand that.")
#                 continue
#             if num in group.index or num < 0:
#                 break
#             else:
#                 print("Sorry, I didn't understand that.")

#     else:
#         num = group.index

#     if num < 0:
#         break
#     else:
#         pkls.loc[num, 'use_this'] = True


# Alternative routine for automatic decision of which pkl files should be used: keep the newest one
recent_pkls_indices = pkls.groupby(['path'], sort=False).tmod.nlargest(1).index.get_level_values(1)
recent_pkls = pkls.loc[recent_pkls_indices.values,:]

In [ ]:
pkl = f'{recent_pkls.path.values[0]}/{recent_pkls.name.values[0]}'
dset = dataset(pkl)